In [4]:
import random
from datasets import load_dataset
import pandas as pd

In [8]:
num_files = 5535

num_files_to_sample = num_files * 0.001
# num_files_to_sample

file_numbers = random.sample(range(0,num_files), int(num_files_to_sample))
# file_numbers

# just a quick reminder of what the number formatting looks like
# number = 35
# f"{number:05d}"

pattern = "data/train-{:05d}-of-05534-*"
file_patterns = [pattern.format(x) for x in file_numbers]
file_patterns

['data/train-01997-of-05534-*',
 'data/train-02679-of-05534-*',
 'data/train-00610-of-05534-*',
 'data/train-04882-of-05534-*',
 'data/train-05270-of-05534-*']

In [9]:

data_sample = load_dataset(
    "tiiuae/falcon-refinedweb",
    data_files=file_patterns, ignore_verifications=True
)
df = pd.DataFrame(data_sample["train"])



Generating train split:   0%|          | 0/968000015 [00:00<?, ? examples/s]

In [14]:
#%%
name = '-'.join([str(x) for x in file_numbers])
df.to_parquet(f'refinedweb_{name}.parquet')

In [15]:
name

'1997-2679-610-4882-5270'

In [16]:
# To use saved file, specify the name here
name = '1997-2679-610-4882-5270'
# name = 'refinedweb_4453-1982-3699-5149-508-282-1127-5210-4203-2213-2152-1842-5375-2118-1363-3607-4652-3485-1715-1719-1214-1952'
path = f'refinedweb_{name}.parquet'
df = pd.read_parquet(path) # refinedweb_1997-2679-610-4882-5270.parquet

In [17]:
df

,content,url,timestamp,dump,segment,image_urls
0,Biolustre Renew Daily Conditioning Sealant is ...,http://www.naturallycurly.com/search/brand/nam...,2016-12-02 22:34:18,CC-MAIN-2016-50,1480698540698.78,[]
1,Give a Gift Membership\nWith this gift of natu...,http://www.nature.org/ourinitiatives/regions/n...,2016-12-02 22:20:47,CC-MAIN-2016-50,1480698540698.78,[[/cs/groups/webcontent/@web/documents/media/g...
2,"As noted in the description, my book chronicle...",http://www.navigatinglifesroadways.com/?page_i...,2016-12-02 22:13:04,CC-MAIN-2016-50,1480698540698.78,[]
3,Bulls @ Pacers\nthe bulls were a very competit...,http://www.nbadraft.net/forum/bulls-pacers,2016-12-02 22:49:35,CC-MAIN-2016-50,1480698540698.78,[]
4,"LAKE FOREST, Calif. — After years of ministeri...",http://www.nbcnews.com/id/10268413,2016-12-02 22:32:49,CC-MAIN-2016-50,1480698540698.78,[[http://media2.s-nbcnews.com/j/msnbc/Componen...
...,...,...,...,...,...,...
874588,Joachim Low cast aside concerns about Germany'...,https://www.skysports.com/football/news/12023/...,2021-07-24 05:08:16,CC-MAIN-2021-31,1627046150129.50,[[https://e0.365dm.com/21/06/768x432/skysports...
874589,At 10:30pm on the 30 of August I boarded QF74 ...,https://www.sleepingwithmyeyesopen.com/2010/09...,2021-07-24 03:27:41,CC-MAIN-2021-31,1627046150129.50,[]
874590,展开查看详情\n1. Today’s Papers • Chord: A Scalable ...,https://www.slidestalk.com/u3808/ChordTapestry16,2021-07-24 04:09:20,CC-MAIN-2021-31,1627046150129.50,[]
874591,A Year of Local Music: Wes Johnson of Archive ...,https://www.slugmag.com/music/interviews/music...,2021-07-24 03:31:10,CC-MAIN-2021-31,1627046150129.50,[]


In [19]:
len(df), df.memory_usage(index=True).sum() * 1e-6

(874593, 41.980592)

In [20]:
import tldextract
# === GET DOMAIN AND SUBDOMAIN ===
domains = []
subdomains = []
full = []
for url in df.url.values:
    extract = tldextract.extract(url)
    domains.append(extract.domain + '.' + extract.suffix)
    subdomains.append(extract.subdomain)
    full.append(extract.subdomain + '.' + extract.domain + '.' + extract.suffix)
df["domain"] = domains
df["subdomain"] = subdomains
df["full"] = full
df['totaldocuments'] = 1

In [21]:
#%%
# === COUNT WORDS === 
df['totalwords'] = [len(x.split()) for x in df['content'].tolist()]

In [22]:
df.describe()

,timestamp,totaldocuments,totalwords
count,874593,874593.0,874593.000000
mean,2018-02-15 18:05:20.751994624,1.0,422.783622
min,2014-03-07 08:07:10,1.0,0.000000
25%,2016-10-21 00:58:00,1.0,99.000000
50%,2018-01-16 10:04:55,1.0,209.000000
75%,2019-11-12 08:12:29,1.0,459.000000
max,2022-05-16 10:29:11,1.0,96563.000000
std,NaN,0.0,1041.974827
